``` docker compose --profile default -f docker-compose.yaml up -d```

In [ ]:
import druidapi
import pandas as pd
import matplotlib.pyplot as plt
import configparser

druid_host = "http://localhost:8888"

druid_headers = {'Content-Type': 'application/json'}

print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)
display = druid.display
sql_client = druid.sql
status_client = druid.status
druid_headers = {'Content-Type': 'application/json'}

config = configparser.ConfigParser()
config.read('auth.ini')

status_client.version

In [ ]:
sql = f'''
SELECT
  TIME_EXTRACT("__time",'hour') AS "hour",
  COUNT (DISTINCT "hex_ident_val") AS "aircraft tracked"
FROM "adsb-json"
WHERE "__time" >= '2024-09-01T00:00:00.000Z'
GROUP BY 1
'''

response = sql_client.sql_query(sql)
df = pd.DataFrame(response.json)

df.plot(x='hour', y='aircraft tracked', marker='o')
plt.xticks(rotation=45, ha='right')
plt.ylabel("Aircraft Tracked")
plt.xlabel("Hour")
plt.gca().get_legend().remove()
plt.show()

In [ ]:
sql = f'''
SELECT
"segment_id", "num_rows"
FROM "sys"."segments" WHERE "datasource"='adsb-json' ORDER BY 1
'''

response = sql_client.sql_query(sql)
df = pd.DataFrame(response.json)

df["segment_id"] = df['segment_id'].str[10:20] + " " + df['segment_id'].str[35:45]

df.plot(kind='bar', x='segment_id', y='num_rows')
plt.xticks(rotation=45, ha='right')
plt.ylabel("Segment")
plt.ylabel("Number of rows")
plt.gca().get_legend().remove()
plt.show()